# Decision tree

설정한 여러가지 조건을 토대로 데이터를 쪼개나가는 방법이다. 예를 들어 "공부 시간이 10시간 이상"인지 아닌지로 데이터를 쪼갠다.

## 1. 정의

- P개의 feature($X_1,X_2,...,X_P$)로 구성된 공간을 J개의 겹치지 않는 구역($R_1,R_2,...,R_J$)으로 나눈다.
- $R_J$ 구역의 예측 값은 그 구역의 mean 값이다
- RSS를 최소화하는 구역, 즉 데이터들의 Y값과 구역의 예측값의 차이를 최소화하는 것이 목표

$$
\text{RSS} = \sum_{j=1}^J\sum_{i \in R_j}(Y^{(i)} - \hat{Y_{R_j}})^2
$$

## 2. 방법

- top-down, greedy approach -> recursive binary splitting
    + top-down: P개의 feature로 이뤄진 공간을 위에서부터 아래로 쪼개나가는 방식(트리의 모양을 생각)
    + Greedy: 현재 상태에서 가장 최선의 선택을 하는 것. 첫 번째 쪼개는 feature가 최선이 아니더라도 그 순간에서 최선의 선택을 해나간다. 이후의 조합에 대해선 신경쓰지 않겠다는 의미
- 순서
    + feature 하나를 정하고, RSS를 가장 크게 줄이는 cutpoint를 정한다.
    + cutpoint로 구역을 두 개 나누고, 나눠진 구역 모두에서 계속 같은 작업을 반복한다.
    + 반복 종료 조건: 한 구역에 속한 데이터가 특정 개수 이하일 때, RSS가 더 이상 작아지지 않을 때까지

## 3. Cost complexity pruning

- 2의 방법까지만 하면 overfitting 문제 발생할 수 있다. 데이터에 대한 이해 없이 반복 종료 조건을 설정하면 굳이 더 이상 쪼갤 필요 없는데도 계속 데이터를 쪼개나갈 수 있기 때문이다.
- 그래서 Pruning, 가지치기 방법을 사용한다.
- 순서
    + 설정한 반복 종료 조건에 맞춰서 일단 최대한 크게 트리를 만든다. 이 트리를 $T_0$라 부른다.
    + 가지를 친다. 즉 가장 데이터를 잘 분류하는, RSS가 가장 작은 subtree를 구한다.

$$
\underset{T}{\arg\min}_T \sum_{m=1}^{|T|}\sum_{i \in R_m}(Y^{(i)} - \hat{Y_{R_j}})^2 + \alpha|T|
$$

> `|T|` : 구역의 개수, 즉 terminal node(leaf)의 개수를 의미

- 위 공식처럼 가지의 개수를 페널티로 줘서 가장 적합한 subtree를 구한다.
- 먼저 알파 값은 적당한 양수 값을 하나 정해서 고정하고 일단 적합한 |T|부터 찾는다.
- 알파값은 나중에 cross-validation을 통해서 찾아낸다.

## 4. Metric

$$
H(S) = -\sum_{c \in C}p\log{(p(c))} \\
I = H(S) - \sum_{i \in \{L,R\}}{|S^i| \over |S|}H(S^i)
$$

- Entropy : 복잡도를 의미
- IG(Information Gain) : 어떤 attribute를 선택했을 때 얻게되는 entropy의 감소 정도

In [3]:
# Entropy 구하기

import math

def calculate_entropy(nums):
    p1 = nums[0] / sum(nums)
    p2 = nums[1] / sum(nums)
    return -p1 * math.log(p1, 2) - p2 * math.log(p2, 2)

def calculate_IG(H, A1, A2):
    H_en = calculate_entropy(H)
    A1_en = calculate_entropy(A1)
    A2_en = calculate_entropy(A2)
    return H_en - (sum(A1)/sum(H)*A1_en + sum(A2)/sum(H)*A2_en)

result = calculate_IG([10, 20], [3, 14], [7, 6])
print('result:', result)

result: 0.1058468751414936


## 5. Overfitting 피하기

![ID3-overfitting](http://slideplayer.com/8111708/25/images/34/Overfitting+As+ID3+adds+new+nodes+to+grow+the+decision+tree%2C+the+accuracy+of+the+tree+measured.+over+the+training+examples+increases+monotonically..jpg)

- Decision tree의 가장 큰 문제점은 overfitting : ID3 알고리즘은 attribute를 다 소모하거나, 모든 노드의 example들이 pure해질 때까지 노드를 추가해나가기 때문에 overfitting이 발생한다.
- node가 많아질수록 training data에 대한 accuracy는 높아지지만 test에 대해선 낮아진다.
- 위와 같은 문제를 해결하기 위해 두 가지 방법을 사용한다.

### 5.1 Setting minimum IG

- IG(Information Gain)이 크지 않은 노드는 더 이상 분리하지 않는다.
- 예를 들어 10,000개 데이터 중에서 9,995개가 A고, 5개만 B라면 더 이상 분리하지 않는다.

### 5.2 Pruning

- 일단 ID3 알고리즘으로 가능한 최대 크기로 tree를 만든다.
- 데이터셋을 train, validation, test 셋으로 나눈다.
    + 100개 데이터 -> 50 / 30, 20
    + validation이 필요한 이유는 y label이 없는 test 데이터셋에 decision tree를 적용하기 전에 tree가 제대로 동작하는지 확인하기 위해서다.
    + 아래 추가로 설명하듯이 여러가지의 sub tree 중 하나를 골라야하기 때문
- 전체 tree에서 subtree를 하나씩 없애본다. parent node만 남기고 child node를 없애보는 것
- subtree를 없애본 모든 시도에 대해서 각각을 validation 데이터셋에 적용해보고 accuracy를 측정한다.
- 가장 높은 accuracy를 보인 subtree를 선택